In [1]:
import abc
import collections
from collections.abc import Sized
from random import shuffle
import random

## Interfaces and Protocols in Python Culture

***Protocols** are defined as the informal interfaces that make polymorphism work in languages with dynamic typing
like Python*.

How do interfaces work in a dynamic-typed language? First, the basics: even without an interface keyword in the anguage, and regardless of ABCs, every class has an interface: the set public attributes (methods or data attributes) implemented or inherited by the class. This includes special methods, like `__getitem__` or `__add__`.

By definition, protected and private attributes are not part of an interface, even if “protected” is merely a naming convention and private attributes are easily accessed.

On the other hand, it’s not a sin to have public data attributes as part of the interface of an object, because—if necessary—a data attribute can always be turned into a property implementing `getter`/`setter` logic without breaking client code that uses the plain `obj.attr` syntax.

A useful complementary definition of **interface** is: *the subset of an object’s public methods that enable it to play a specific role in the system*.

Protocols are interfaces, but because they are informal—defined only by documentation and conventions—protocols cannot be enforced like formal interfaces can. A protocol may be partially implemented in a particular class, and that’s OK.

## Python Digs Sequences

Now, take a look at the Foo class in Example 11-3. It does not inherit from `abc.Sequence`, and it only implements one method of the sequence protocol: `__getitem__` (`__len__` is missing).

In [2]:
class Foo:

    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [3]:
f = Foo()
f[1]

10

In [4]:
for i in f: print(i)

0
10
20


In [5]:
20 in f

True

There is no method `__iter__` yet Foo instances are iterable because—as a fallback—when Python sees a `__getitem__` method, it tries to iterate over the object by calling that method with integer indexes starting with 0. Because Python is smart enough to iterate over Foo instances, it can also make the in operator work even if Foo has no `__contains__` method: it does a full scan to check if an item is present.

In summary, given the importance of the sequence protocol, in the absence `__iter__` and `__contains__` Python still manages to make iteration and the in operator work by
invoking `__getitem__`.

Our original FrenchDeck from Chapter 1 does not subclass from abc.Sequence either, but it does implement both methods of the sequence protocol: `__getitem__` and
`__len__`.

In [6]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [7]:
class FrenchDeck:
    
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

Iteration in Python represents an extreme form of duck typing: the interpreter tries two different methods to iterate over objects.

## Monkey-Patching to Implement a Protocol at Runtime
The FrenchDeck class from Example 11-4 has a major flaw: it cannot be shuffled.

In [8]:
l = list(range(10))
shuffle(l)

In [9]:
l

[8, 7, 3, 2, 0, 5, 4, 1, 9, 6]

In [10]:
deck = FrenchDeck()
try:
    shuffle(deck)
except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")

TypeError: 'FrenchDeck' object does not support item assignment


The problem is that shuffle operates by swapping items inside the collection, and FrenchDeck only implements the immutable sequence protocol. Mutable sequences must also provide a `__setitem__` method.

### Monkey patching FrenchDeck to make it mutable and compatible with `random.shuffle`

In [11]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [12]:
FrenchDeck.__setitem__ = set_card
shuffle(deck)

In [13]:
deck[:5]

[Card(rank='Q', suit='spades'),
 Card(rank='J', suit='clubs'),
 Card(rank='5', suit='hearts'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='diamonds')]

***Monkey patching:** changing a class or module at runtime, without touching the source code*. Monkey patching is powerful, but the code that does the actual patching is very tightly coupled with the program to be patched, often handling private and undocumented parts.

Besides being an example of monkey patching, Example 11-6 highlights that protocols are dynamic: random.shuffle doesn’t care what type of argument it gets, it only needs
the object to implement part of the mutable sequence protocol.

***Goose typing:** isinstance(obj, cls) is now just fine… as long as cls is an abstract base class—in other words, cls’s metaclass is abc.ABCMeta*.

Python’s ABCs add one major practical advantage: the register class method, which lets end-user code “declare” that a certain class becomes a “virtual” subclass of an ABC.

Sometimes you don’t even need to register a class for an ABC to recognize it as a subclass!

That’s the case for the ABCs whose essence boils down to a few special methods. For example:

In [14]:
class Struggle:
    
    def __len__(self): return 23


isinstance(Struggle(), Sized)

True

As you see, abc.Sized recognizes Struggle as “a subclass,” with no need for registration, as implementing the special method named __len__ is all it takes.

In several classes in this book, when I needed to take a sequence of items and process them as a list, instead of requiring a list argument by type checking, I simply took the argument and immediately built a list from it: that way I can accept any iterable, and if the argument is not iterable, the call will fail soon enough with a very
clear message. One example of this code pattern is in the __init__ method in Example 11-13, later in this chapter. Of course, this approach wouldn’t work if the sequence
argument shouldn’t be copied, either because it’s too large or because my code needs to change it in place. Then an insinstance(x, abc.MutableSequence) would be better. If any iterable is acceptable, then calling iter(x) to obtain an iterator would be the way to go, as we’ll see in “Why Sequences Are Iterable: The iter Function” on page 404.

## Subclassing an ABC

In [15]:
class FrenchDeck2(collections.abc.MutableSequence):

    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value): #
        self._cards[position] = value

    def __delitem__(self, position): #
        del self._cards[position]

    def insert(self, position, value): #
        self._cards.insert(position, value)

Python does not check for the implementation of the abstract methods at import time (when the frenchdeck2.py module is loaded and compiled), but only at runtime when
we actually try to instantiate FrenchDeck2. Then, if we fail to implement any abstract method, we get a TypeError exception with a message such as "Can't instantiate abstract class FrenchDeck2 with abstract methods `__delitem__`, insert".

## Defining and Using an ABC

In [16]:
class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """
    
    def loaded(self):
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

An ABC may include concrete methods. Concrete methods in an ABC must rely only on the interface defined by the ABC (i.e., other concrete or abstract methods or properties of the ABC).

An abstract method can actually have an implementation. Even if it does, subclasses will still be forced to override it, but they will be able to invoke the abstract method with super(), adding functionality to it instead of implementing from scratch.

### A fake Tombola doesn’t go undetected

In [17]:
class Fake(Tombola):
    
    def pick(self):
        return 13

In [18]:
Fake

__main__.Fake

In [19]:
try:
    f = Fake()
except Exception as e:
    print(f"{type(e).__name__}: {str(e)}")

TypeError: Can't instantiate abstract class Fake with abstract methods load


TypeError is raised when we try to instantiate Fake. The message is very clear: Fake is considered abstract because it failed to implement load, one of the abstract methods declared in the Tombola ABC.

***The best way to declare an ABC is to subclass `abc.ABC` or any other ABC.***

In [20]:
# Stacking decorators
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_method(cls):
        pass

## Subclassing the Tombola ABC

In [21]:
class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('Pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [22]:
class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls - list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('Pick from empty BingoCage')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        returntuple(sorted(self._balls))

## A virtual subclass of Tombola

An essential characteristic of goose typing—and the reason why it deserves a waterfowl name—is the ability to register a class as a virtual subclass of an ABC, even if it does not inherit from it. When doing so, we promise that the class faithfully implements the interface defined in the ABC—and Python will believe us without checking. If we lie,
we’ll be caught by the usual runtime exceptions.

This is done by calling a register method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will be recognized as such by functions like
issubclass and isinstance, but it will not inherit any methods or attributes from the ABC.

Virtual subclasses do not inherit from their registered ABCs, and are not checked for conformance to the ABC interface at any time, not even when they are instantiated. It’s up to the subclass to actually implement all the methods needed to avoid runtime errors.

The register method is usually invoked as a plain function, but it can also be used as a decorator.

In [23]:
@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList.')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [24]:
issubclass(TomboList, Tombola)

True

In [25]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [26]:
Tombola.__mro__

(__main__.Tombola, abc.ABC, object)

## Geese Can Behave as Ducks
A class can be recognized as a virtual subclass of an ABC even without registration.

In [27]:
class Struggle:
    def __len__(self): return 23

In [28]:
isinstance(Struggle(), Sized)

True

In [29]:
issubclass(Struggle, Sized)

True

Class Struggle is considered a subclass of `abc.Sized` by the issubclass function (and, consequently, by isinstance as well) because abc.Sized implements a special class method named `__subclasshook__`.

In [30]:
# Sized definition from the source code
class Sized(metaclass=abc.ABCMeta):
    __slots__ = ()
    
    @abc.abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

Is it a good idea to implement `__subclasshook__` in our own ABCs? Probably not. All the implementations of `__subclasshook__` I’ve seen in the Python source code are in ABCs like Sized that declare just one special method, and they simply check for that special method name.